In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
gc.enable()

In [3]:
steps = 16
height = 108
width = 108

In [4]:
#change the path to where you save the input numpy arrays
path = 'E:/notes_database/32_frames_108/'
lstm = np.array(glob.glob(path+'lstm_inputs/*'))
cnn = np.array(glob.glob(path+'cnn_inputs/*'))
y_train = np.array(glob.glob(path+'y_train/*'))
c_lstm = np.array(glob.glob(path+'counter_lstm_inputs/*'))
c_cnn = np.array(glob.glob(path+'counter_cnn_inputs/*'))
c_y_train = np.array(glob.glob(path+'counter_y_train/*'))

In [5]:
#The function analyzes the actual accuracy
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if abs(prediction[x][y] - y_test[x][y]) > 0.5:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [6]:
def inputs_to_model(order, num, start, ending):
    cnn_inputs = np.load(cnn[order[num][0]])
    labels_inputs = np.load(y_train[order[num][0]])
    lstm_inputs = np.load(lstm[order[num][0]])
    
    lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[order[num][0]]), axis=0)
    cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[order[num][0]]), axis=0)
    labels_inputs = np.append(labels_inputs, np.load(c_y_train[order[num][0]]), axis=0)
    
    for file in order[num][start+1:ending]:       
        lstm_inputs = np.append(lstm_inputs, np.load(lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(c_y_train[file]), axis=0)
    
    lstm_inputs = lstm_inputs.astype(np.float16)
    labels_inputs = labels_inputs.astype(np.float16)
    return lstm_inputs, cnn_inputs, labels_inputs

In [ ]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (4,4), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.2)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [8]:
def main_2(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [9]:
def main_3(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [10]:
def train_test_whole(classifier, num, name, epochs=100, batchsize=32):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(train_order, num, 0, train_order[num].shape[0]+1)
    
    classifier.fit([cnn_inputs, lstm_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, lstm_test], labels_test))
    
    #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
    #print('val_loss: ', CV_loss)
    #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [11]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [12]:
lstm_test, cnn_test, labels_test = inputs_to_model(test_order, 0, 0, test_order[0].shape[0]+1)

In [13]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main_2(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-32units108-model1.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys


Train on 139086 samples, validate on 70356 samples
Epoch 1/100
139086/139086 [==============================] - 223s 2ms/step - loss: 0.0556 - acc: 0.9817 - val_loss: 0.0535 - val_acc: 0.9822
Epoch 2/100
139086/139086 [==============================] - 218s 2ms/step - loss: 0.0329 - acc: 0.9897 - val_loss: 0.0336 - val_acc: 0.9895
Epoch 3/100
139086/139086 [==============================] - 219s 2ms/step - loss: 0.0271 - acc: 0.9914 - val_loss: 0.1738 - val_acc: 0.9761
Epoch 4/100
139086/139086 [==============================] - 219s 2ms/step - loss: 0.0234 - acc: 0.9925 - val_loss: 0.0506 - val_acc: 0.9845
Epoch 5/100
139086/139086 [==============================] - 219s 2ms/step - loss: 0.0207 - acc: 0.9933 - val_loss: 0.0245 - val_acc: 0.9927
Epoch 6/100
139086/139086 [==============================] - 219s 2ms/step - loss: 0.0188 - acc: 0.9938 - val_loss: 0.1013 - val_acc: 0.9708
Epoch 7/100
139086/139086 [==============================] - 215s 2ms/step - loss: 0.0172 - acc: 0.9943

139086/139086 [==============================] - 212s 2ms/step - loss: 0.0060 - acc: 0.9980 - val_loss: 0.0407 - val_acc: 0.9916
Epoch 59/100
 20672/139086 [===>..........................] - ETA: 2:40 - loss: 0.0054 - acc: 0.9982

KeyboardInterrupt: 

In [14]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-32units108-model1.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  


Train on 139086 samples, validate on 70356 samples
Epoch 1/100
139086/139086 [==============================] - 250s 2ms/step - loss: 0.0666 - acc: 0.9778 - val_loss: 0.0381 - val_acc: 0.9880
Epoch 2/100
139086/139086 [==============================] - 245s 2ms/step - loss: 0.0380 - acc: 0.9879 - val_loss: 0.0379 - val_acc: 0.9882
Epoch 3/100
139086/139086 [==============================] - 244s 2ms/step - loss: 0.0332 - acc: 0.9895 - val_loss: 0.0285 - val_acc: 0.9912
Epoch 4/100
139086/139086 [==============================] - 244s 2ms/step - loss: 0.0306 - acc: 0.9904 - val_loss: 0.2294 - val_acc: 0.9593
Epoch 5/100
139086/139086 [==============================] - 245s 2ms/step - loss: 0.0287 - acc: 0.9909 - val_loss: 0.0306 - val_acc: 0.9906
Epoch 6/100
 79264/139086 [================>.............] - ETA: 1:30 - loss: 0.0273 - acc: 0.9914

KeyboardInterrupt: 